In [34]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from dotenv import load_dotenv
import os

In [35]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC','VIAC','WLTW'])]
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [36]:
# Load .env environment variables
load_dotenv()

True

In [37]:
IEX_CLOUD_API_TOKEN = os.getenv("IEX_CLOUD_API_SECRET_TOKEN")
type(IEX_CLOUD_API_TOKEN)

str

In [38]:
#Making Our First API Call
#Market Capitalization for each stock
#Price of each stock

symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com//stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)


{'avgTotalVolume': 64133372, 'calculationPrice': 'close', 'change': -1.09, 'changePercent': -0.00586, 'close': 184.92, 'closeSource': 'official', 'closeTime': 1686945600459, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 185, 'delayedPriceTime': 1686945479741, 'extendedChange': 0.03, 'extendedChangePercent': 0.00016, 'extendedPrice': 184.95, 'extendedPriceTime': 1686959999000, 'high': 184.92, 'highSource': 'Close', 'highTime': 1686945600459, 'iexAskPrice': None, 'iexAskSize': None, 'iexBidPrice': None, 'iexBidSize': None, 'iexClose': 185.02, 'iexCloseTime': 1686945599694, 'iexLastUpdated': None, 'iexMarketPercent': None, 'iexOpen': 186.735, 'iexOpenTime': 1686922200065, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexVolume': None, 'lastTradeTime': 1686945599917, 'latestPrice': 184.92, 'latestSource': 'Close', 'latestTime': 'June 16, 2023', 'latestUpdate': 1686945600459, 'latestVolume': 101256225, 'low': 184.92, 'lowSource': 'Close', 'lowTime': 1686945600000, 'ma

In [39]:
#parce price
#parce market cap
data['latestPrice']

data['marketCap']



2908551573840

In [40]:
#Scale all the process to all the stocks
#Data to a Pandas DataFrame
#0000 row added for visual effect
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of shares to Buy']
final_dataframe = pd.DataFrame([[0,0,0,0,]],columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of shares to Buy
0,0,0,0,0


In [41]:
final_dataframe = final_dataframe.append(
                                        pd.Series(['Ticker',
                                                    data['latestPrice'],
                                                    data['marketCap'],
                                                    'N/A'],
    #which columns to add this data too
                                                    index = my_columns),
    #Necessary when appending data to pandas data frame
                                        ignore_index=True)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of shares to Buy
0,0,0.00,0,0
1,Ticker,184.92,2908551573840,N/A


In [42]:
#loop through the tickers in our list of stocks
# final_dataframe = pd.DataFrame(columns = my_columns)
# for symbol in stocks['Ticker']:
#     api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
#     #this line is very slow
#     data = requests.get(api_url).json()
#     #final_dataframe.append( Was the initial try
#     final_dataframe = final_dataframe.append(
#                                         pd.Series([symbol,
#                                                     data['latestPrice'],
#                                                     data['marketCap'],
#                                                     'N/A'],
#                                                 index = my_columns),
#                                         ignore_index = True)

In [43]:
#using Batch API calls to improve performance
def chunks(lst, n):
    """Yield successive n-sized chunks from 1st."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [44]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series([symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['marketCap'],
                    'N/A'],
                index = my_columns),
            ignore_index = True
        )
final_dataframe   

,Ticker,Stock Price,Market Capitalization,Number of shares to Buy
0,A,121.13,35778872834,N/A
1,AAL,16.48,10759792000,N/A
2,AAP,69.22,4114703435,N/A
3,AAPL,184.92,2908551573840,N/A
4,ABBV,138.64,244601121235,N/A
...,...,...,...,...
496,YUM,138.58,38814454104,N/A
497,ZBH,144.10,30054833969,N/A
498,ZBRA,282.40,14523927451,N/A
499,ZION,28.63,4240123070,N/A


In [46]:
#Calculating the number of Shares to Buy #start by creating an input
#try and except is to prevent strings from breaking it
portfolio_size = input('Enter the value of your portfolio:')

try:
    val = float(portfolio_size)
    print(val)
except:
    print("That's not a number! \nPlease try again:")
    portfolio_size = input('Enter the value of your portfolio:')
    val = float(portfolio_size)

Enter the value of your portfolio: 100000


100000.0


In [48]:
postion_size = val/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(postion_size/final_dataframe.loc[i, 'Stock Price'])
final_dataframe
#print(position_size)
#number_of_apple_shares = position_size/500
#Math to round down
#print(math.floor(number_of_apple_shares))

,Ticker,Stock Price,Market Capitalization,Number of shares to Buy,Number of Shares to Buy
0,A,121.13,35778872834,N/A,1.0
1,AAL,16.48,10759792000,N/A,12.0
2,AAP,69.22,4114703435,N/A,2.0
3,AAPL,184.92,2908551573840,N/A,1.0
4,ABBV,138.64,244601121235,N/A,1.0
...,...,...,...,...,...
496,YUM,138.58,38814454104,N/A,1.0
497,ZBH,144.10,30054833969,N/A,1.0
498,ZBRA,282.40,14523927451,N/A,0.0
499,ZION,28.63,4240123070,N/A,6.0


In [50]:
#creating the formats we'll Need for our .xlsx file
writer = pd.ExcelWriter('recommended trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [51]:
#creating the formats we'll need for our .xlsx file
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color' : background_color,
        'border' : 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color' : background_color,
        'border' : 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format' : '$0.00',
        'font_color': font_color,
        'bg_color' : background_color,
        'border' : 1
    }
)

In [52]:
#Applying the formats to the columns of our .xlsx file
#writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
#writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
#writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
#writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)

#writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
#writer.sheets['Recommended Trades'].write('B1', 'Stock Price', dollar_format)
#writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', dollar_format)
#writer.sheets['Recommended Trades'].write('D1', 'Ticker', integer_format)

In [53]:
column_formats = {
    'A':['Ticker', string_format],
    'B':['Stock Price' , dollar_format],
    'C':['Market Capitalization', dollar_format],
    'D':['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    #Two Loops for previous to clean cold
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
writer.save()